In [65]:
import requests
import snowflake.connector

HOST = "FRHSAXD-CFB18747.snowflakecomputing.com"
API_ENDPOINT = "/api/v2/cortex/agent:run"
API_TIMEOUT = 50000  # in milliseconds

CORTEX_SEARCH_SERVICES = "DASH_DB.DASH_SCHEMA.VEHICLES_INFO"
SEMANTIC_MODELS_SUPPLY_CHAIN = "@DASH_DB.DASH_SCHEMA.DASH_SEMANTIC_MODELS/supply_chain_semantic_model.yaml"
SEMANTIC_MODELS_SUPPORT_TICKETS = "@DASH_DB.DASH_SCHEMA.DASH_SEMANTIC_MODELS/support_tickets_semantic_model.yaml"

In [ ]:
snow_conn = snowflake.connector.connect(
    user="ANGELCORTEX2025",
    password="",
    account="XIB99769",
    host=HOST,
    port=443,
    warehouse="COMPUTE_WH",
    role="ACCOUNTADMIN",
)


with snow_conn.cursor() as cur:
    df = cur.execute("SELECT CURRENT_ACCOUNT(), CURRENT_ROLE(), CURRENT_USER()").fetch_pandas_all()

df

,CURRENT_ACCOUNT(),CURRENT_ROLE(),CURRENT_USER()
0,XIB99769,ACCOUNTADMIN,ANGELCORTEX2025


In [175]:
import requests
import json
import pprint

def send_question(question):
    payload = {
        "model": "llama3.1-70b",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": question
                    }
                ]
            }
        ],
        "tools": [
            { "tool_spec": { "type": "cortex_search", "name": "vehicles_info_search" } },
            { "tool_spec": { "type": "cortex_analyst_text_to_sql", "name": "support" } },
            { "tool_spec": { "type": "cortex_analyst_text_to_sql", "name": "supply_chain" } }
        ],
        "tool_resources": {
            "supply_chain": {"semantic_model_file": SEMANTIC_MODELS_SUPPLY_CHAIN},
            "support": {"semantic_model_file": SEMANTIC_MODELS_SUPPORT_TICKETS},
            "vehicles_info_search": {
                "name": CORTEX_SEARCH_SERVICES,
                "max_results": 3,
                "title_column": "title",
                "id_column": "relative_path"
            }
        }
    }

    response = requests.post(
        url=f"https://{HOST}/{API_ENDPOINT}",
        json=payload,
        headers={
            "Authorization": f'Snowflake Token="{snow_conn.rest.token}"',
            "Content-Type": "application/json",
            "Accept": "application/json",
        },
        stream=True
    )
    return response

def parse_response_to_events(response):
    """Parse raw SSE response into a list of events"""
    events = []
        
    for line in response.iter_lines():
        if line:
            decoded = line.decode('utf-8')
            
            if decoded.startswith("data: "):
                content = decoded.replace("data: ", "")
                if content == "[DONE]":
                    break

                parsed = json.loads(content)
                events.append(parsed)

    return events

def process_sse_response(response):
    """Process SSE response"""
    text = ""
    sql = ""
    citations = []
    
    if not response:
        return text, sql, citations
    if isinstance(response, str):
        return text, sql, citations
    try:
        for event in response:
            if event.get('object') == "message.delta":
                delta = event.get('delta', {})
                
                for content_item in delta.get('content', []):
                    content_type = content_item.get('type')
                    if content_type == "tool_results":
                        tool_results = content_item.get('tool_results', {})
                        if 'content' in tool_results:
                            for result in tool_results['content']:
                                if result.get('type') == 'json':
                                    text += result.get('json', {}).get('text', '')
                                    search_results = result.get('json', {}).get('searchResults', [])
                                    for search_result in search_results:
                                        citations.append({'source_id':search_result.get('source_id',''), 'doc_id':search_result.get('doc_id', ''), 'text':search_result.get('text', '')})
                                    sql = result.get('json', {}).get('sql', '')
                    if content_type == 'text':
                        text += content_item.get('text', '')
                            
    except json.JSONDecodeError as e:
        print(f"Error processing events: {str(e)}")
                
    except Exception as e:
        print(f"Error processing events: {str(e)}")
        
    return text, sql, citations

In [176]:
response = send_question("What is the latest, most effective way to recycle rubber tires?")
events = parse_response_to_events(response)

for event in events:
    print(event)

{'id': 'msg_001', 'object': 'message.delta', 'delta': {'content': [{'index': 0, 'type': 'tool_use', 'tool_use': {'tool_use_id': 'toolu_f51e7b74', 'name': 'vehicles_info_search', 'input': {'query': 'What is the latest, most effective way to recycle rubber tires?', 'columns': ['relative_path', 'title'], 'filters': '', 'limit': 3.0, 'requestId': 'bfb54a79-54a6-46d5-aa34-ea27238ddcd1', 'scoringConfig': '<nil>', 'experimentalJson': ''}}}, {'index': 0, 'type': 'tool_results', 'tool_results': {'tool_use_id': 'toolu_f51e7b74', 'content': [{'type': 'json', 'json': {'searchResults': [{'source_id': 1.0, 'doc_id': 'Snowtires_Automotive_ESG_Recycling_Policy_Full.pdf', 'doc_title': 'Snowtires_Automotive_ESG_Recycling_Policy_Full', 'text': '## Recycling Policy Overview\nAs part of our commitment to sustainability, Snowtires Automotive has developed robust recycling policies. Our tire recycling programs aim to recover materials, minimizing environmental impact and reducing the need for virgin resource

In [177]:
# Pass the parsed events to the process_sse_response function
text, sql, citations = process_sse_response(events)

# Print the results
print("Text:", text)
print("SQL:", sql)
print("Citations:")

for citation in citations:
    print(citation)

Text: According to Snowtires Automotive's recycling policy, tires with over 60% rubber content are processed to reclaim up to 80% of the rubber for reuse in new tires and other products, significantly reducing waste and reliance on virgin rubber 【†1†】.
SQL: 
Citations:
{'source_id': 1.0, 'doc_id': 'Snowtires_Automotive_ESG_Recycling_Policy_Full.pdf', 'text': '## Recycling Policy Overview\nAs part of our commitment to sustainability, Snowtires Automotive has developed robust recycling policies. Our tire recycling programs aim to recover materials, minimizing environmental impact and reducing the need for virgin resources.\n# Recycling Specifications\n\n## High Rubber Content Tires\nTires with over 60% rubber content are processed to reclaim up to 80% of the rubber for reuse in new tires and other products. This significantly reduces waste and reliance on virgin rubber.\n## High Carbon Black Content Tires\nTires with over 30% carbon black content are processed to recover 70% of the carbo